In [1]:
from typing import List, Optional

from sly_llama import llm_call
from langchain.llms import OpenAI

from pydantic import BaseModel

llm = OpenAI()

#### Lets define what the add function does and wrap it in an llm call

In [2]:
@llm_call(llm)
def add(x: str, y: str) -> str:
    """
    calculate {x} + {y}
    only return the number and nothing else
    """

In [3]:
add(1, 2)

'\n3'

In [4]:
add(1, 3) + add(1,1)

'\n4\n2'

#### Problem: strings don't add, lets try again but with ints


In [5]:
@llm_call(llm)
def add(x: str, y: str) -> int:
    """
    calculate {x} + {y}
    only return the number and nothing else
    """

add(1, 3) + add(1,1)

6

Lets make a recipe

In [13]:

@llm_call(llm)
def get_recipe(dish: str, units: str) -> str:
    """
    Write a resipe for this {dish}
    Be sure to include all the ingridients in {units} units.


    ingridients: < neccesary ingridients>
    intructions: < the instructions for making the dish>
    vegan : <this value must be one of [True, False] indicating weather the recipe is vegan>

    """

In [14]:
print(get_recipe("jank", "metric"))


Ingridients: 
- 2 cups of uncooked white rice
- 2 tablespoons of vegetable oil
- 1 onion, chopped
- 2 cloves of garlic, minced
- 2 teaspoons of ground cumin
- 2 teaspoons of chili powder
- 2 tablespoons of tomato paste
- 2 cups of vegetable broth
- 1 bell pepper, chopped
- 1 can (14.5 ounces) of diced tomatoes
- 1 can (15 ounces) of black beans, drained
- 1 teaspoon of salt
- 1/2 teaspoon of black pepper

Instructions:
1. Heat oil in a large pot over medium heat.
2. Add onion and garlic and cook for 1 to 2 minutes, stirring occasionally.
3. Add cumin, chili powder, and tomato paste and cook for 1 minute, stirring constantly.
4. Add the rice and stir to coat.
5. Add the vegetable broth, bell pepper, tomatoes, black beans, salt, and pepper and stir to combine.
6. Increase the heat to high and bring to a boil.
7. Reduce the heat to low and cover. Simmer for 20 minutes, stirring occasionally.
8. Remove from heat and let stand covered for 5 minutes.



#### That's great but what if we want to parse the output to a pydantic class

#### Let define the output class and how we want to parse the llm output

In [15]:
from pydantic import BaseModel

class Recipe(BaseModel):
    ingridients: str | List[str]
    instructions : str | List[str]
    vegan: bool

    @classmethod
    def from_llm_output(cls, llm_output: str):
        recipe = {}
        parts = llm_output.casefold().partition('instructions')
        recipe['ingridients']  = parts[0].replace('ingridients', '').replace('[],"', '').strip().split('\n')
        recipe['instructions'] = parts[2].partition('vegan')[0].replace('[],"', '').strip().split('\n')
        recipe['vegan']  = bool(parts[2].partition('vegan')[1].replace('[],"\n', '').strip())
        return cls.parse_obj(recipe)



#### And ammend the return type

In [16]:
@llm_call(llm)
def get_recipe(dish: str, units: str) -> Recipe:
     """
     Write a resipe for this {dish}
     Be sure to include all the ingridients in {units} units.

     ingridients: < neccesary ingridients>
     intructions: < the instructions for making the dish>
     vegan : <this value must be one of [True, False] indicating weather the recipe is vegan>
     """

In [17]:
recipe = get_recipe('kchapuri', 'metric')
recipe.instructions

[':',
 '1. in a large bowl, mix together the chapathi flour, oil, and salt.',
 '2. slowly add the water and mix together until a dough forms.',
 '3. knead the dough for 5 minutes.',
 '4. cover the dough with a damp cloth and let it rest for 30 minutes.',
 '5. after the dough has rested, divide it into 8 equal parts.',
 '6. using a rolling pin, roll each of the dough pieces into circles.',
 '7. heat 2 tablespoons of oil in a large skillet over medium heat.',
 '8. place one of the circles on the hot skillet and cook for 1 minute on each side.',
 '9. place the chapathi on a plate and serve.']

#### Hmm that was a lot of work and looks like we did not do a good job, lets ask it to give us some juicy JSON

In [18]:
from sly_llama import JsonBaseModel

class Recipe(JsonBaseModel):
    ingridients: str | List[str]
    instructions : str | List[str]
    vegan: bool

#### Llamas are not so good at json so may be let it learn from its mistakes

In [19]:
@llm_call(llm)
def get_recipe(dish: str, units: str, error_message: str) -> Recipe:
    """
    Write a resipe for this {dish}
    Be sure to include all the ingridients in {units} units.

    You should provide your response in JSON Format

    ingridients: < neccesary ingridients>
    intructions: < the instructions for making the dish>
    vegan : <this value must be one of [True, False] indicating weather the recipe is vegan>

    {error_message}
    """

In [20]:
from sly_llama import LlmException

recipe = None
error_message = ''

while not recipe:
    try:
        recipe = get_recipe('kchapuri', 'metric', error_message)
        
    except LlmException as e:
        error_message = e.message
        print(error_message)
recipe

b'' 
 The output was not valid JSON, be sure to only provide JSON


Recipe(ingridients=['1 cup of all-purpose flour', '1/2 teaspoon of salt', '1/2 teaspoon of baking powder', '3 tablespoons of vegetable oil', '3/4 cup of water'], instructions=['In a large bowl, mix together the flour, salt, and baking powder.', 'Add the vegetable oil and water and mix until everything is combined and a ball of dough forms.', 'On a lightly floured surface, knead the dough for 3 minutes.', 'Cover the dough with a damp cloth and let it rest for at least 15 minutes.', 'Roll out the dough to about 1/8 inch thickness.', 'Cut the dough into 8 equal pieces.', 'Roll each piece into a 5 inch circle.', 'Heat a skillet over medium heat and lightly grease with vegetable oil.', 'Once the skillet is hot, place one of the circles in the skillet and cook for about 1 minute.', 'Flip the chapati and cook for an additional minute.', 'Repeat with the remaining circles.'], vegan=True)